In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020'])

In [3]:
file = FILES_TO_ANALYZE['Winterthur 2020']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='KSW', year='2020', sheets=['Änderungen_Winterthur_2020'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 214 cases for KSW 2020
41/214 rows were deleted because contained NaNs
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [5]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,20049284027,E66EFC8932E6E4BF,M,66,25,3,H01B,C241,M200,K830,<NA>,"009a13::20200310,009b11::20200310,99b711::2020...","009A13::20200310,009B11::20200310,99B711::2020..."
1,20134514027,89F52CDD41B3C501,M,73,5,3,F24E,I210,M990,S2240,<NA>,"99b811::20200402,39c111::20200402,39c211::2020...","99B811::20200402,0040::20200402,39C111::202004..."
2,20071661027,C183570C2D12CCAF,M,61,10,3,R61D,C9000,M100,I483,<NA>,<NA>,<NA>
3,20350424027,E93C58A92B1946D2,W,77,10,3,I46D,M167,M200,D689,<NA>,"990a::20200223,99b811::20200223","990A::20200223,99B811::20200223"
4,20359514027,DE96E9E3453EADE4,W,80,6,3,H61B,C220,M100,"K703,L8914",<NA>,"938a12::20200406,938b11::20200404","938A12::20200406,938B11::20200404"


# match to the database


In [6]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['20049284027', '20134514027', '20071661027', '20350424027',
       '20359514027', '20367791027', '20510174027', '20579504027',
       '20635331027', '20278604027', '20409191027', '20411774027',
       '20421731027', '20526191027', '20535404027', '20542931027',
       '20544584027', '20575544027', '20792504027', '20804984027',
       '20806784027', '20907314027', '20956241027', '20983334027',
       '21085301027', '21140504027', '21271661027', '21282461027',
       '21343271027', '21344174027', '21374414027', '21375074027',
       '21576701027', '20780261027', '20877614027', '20890334027',
       '20892584027', '21021944027', '21178664027', '21182774027',
       '21200444027', '21377834027', '21589064027', '21742871027',
       '21774374027', '21819914027', '21838361027', '21849584027',
       '21939614027', '22064054027', '22067114027', '22087391027',
       '22180874027', '22199891027', '22276214027', '22304384027',
       '21318644027', '21617564027', '21705461027', '217881440

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,0.946,451004,13,20030774027,00230D81CCDE5371,59,0,M,10,16,0,0,0,2020-04-30,01,2020-05-10,00,G48C,G48,3
1,3.191,447945,13,20049284027,E66EFC8932E6E4BF,66,0,M,25,4,0,0,0,2020-03-09,01,2020-04-03,00,H01B,H01,3
2,0.799,444578,13,20071661027,C183570C2D12CCAF,61,0,M,10,3,0,0,0,2020-01-26,01,2020-02-05,00,R61D,R61,3
3,0.841,442410,13,20110871027,30D09E9567F27FB6,31,0,W,5,5,0,0,0,2019-12-30,01,2020-01-04,00,O01G,O01,0
4,1.303,449277,13,20134514027,89F52CDD41B3C501,73,0,M,5,16,0,0,0,2020-04-02,01,2020-04-07,00,F24E,F24,3


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 172 out of 173 revised cases from DtoD that are matched with the database for KSW 2020


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match